In [1]:
%reset -f

from functools import reduce
import random
import numpy as np
from itertools import combinations
from itertools import product
from itertools import combinations_with_replacement
from datetime import datetime

from stats import Stats
from character import Character

# File path and name
file_path = "res.txt"
SLOT_NUMER = 37

#============getting all the items from the db.csv file=========================#
items_in_slot = {k:[] for k in range(SLOT_NUMER)}
import pandas as pd
df=pd.read_csv('db.csv')
for index, row in df.iterrows():
    d = row.dropna().to_dict()
    
    items_in_slot[d['slot']].append(Stats(d['slot'], d['name'], d['set'],attributes=d))

#=======filter items_in_slot and prepare for the combination function===========#
items_in_slot_filtered = list(filter(None, items_in_slot.values()))

#================print some info about the objects loaded=======================#
for slot, item in items_in_slot.items():
    if len(item) > 0:
        print(f'slot: {slot:3}, choices:{len(item)}')
items_in_slot_filtered_len = reduce(lambda a, b: a*len(b), [1,*items_in_slot_filtered])
print(f'Items combinations: {items_in_slot_filtered_len}')


#==============base stats for a specific class (not present in db)==============#
base_class_stats    = { 'phy_atk':1202,	'mag_atk':1202,	'phy_def':301,	'mag_def':301,	'hp':157000,
                        'dmg_per':      18,  
                        'dmg_per_m':    0,
                        'crit':         25,        
                        'maxi':         0,        
                        'crit_dmg':     25,

                        'pola':         0,
                        'specific':     0,
                        'all_s_dmg':    3,
                        'boss_dmg':     0,
                        'adapt':        0,
                        'cdr':          0
                       }

#===============================force passives==================================#
head_hunter = {'dmg_per':-24, 'boss_dmg':80}


#============================sockets combinations===============================#
available_sockets = [  
                        Stats('0', 'socket', '', {'maxi':6}),
                        Stats('0', 'socket', '', {'crit':6}),
                        Stats('0', 'socket', '', {'boss_dmg':2.5})
                    ]

sockets_combinations = list(combinations_with_replacement(available_sockets, 38))
sockets_combinations_len = len(sockets_combinations)
print(f'Socket combinations: {sockets_combinations_len}')

# generate item for each socket combination, grouping sockets into a single item
itemized_sockets = []
for i, sockets in enumerate(sockets_combinations):
    tot_socket = Stats('0', 'SOCKETS')
    for socket in sockets:
        tot_socket += socket
    itemized_sockets.append(tot_socket)


#=========================combinatory for all items=============================#
combs = list(product(   *list(items_in_slot_filtered),
                        itemized_sockets  # sockets combinations item
                        )
             )
#combs = random.sample(combs, 100000)
combs = combs[::50000]
combs_len = len(combs)
print(f'Total combinations: {combs_len}')

print(f'Total combinations as expected: {combs_len==(items_in_slot_filtered_len*sockets_combinations_len)}')
#=============================create specific items ============================#
base_class_stats_item = Stats('base','class base stats','',attributes=base_class_stats)
head_hunter_force_passive = Stats('HH','unique HH','',attributes=head_hunter)



#===========equip items combinations and specific items if needed===============#
chars = []
for i, items in enumerate(combs):
    # create a new "character" for each item combination
    new_character = Character('U_%s'%i, 'jeff_%s'%i)
    new_character = new_character.equip(base_class_stats_item, base_class_stats_item.id)
    new_character = new_character.equip(head_hunter_force_passive, head_hunter_force_passive.id)
    
    # equip all items
    for item in items:
        new_character = new_character.equip(item, item.id)
    
    # save to list
    chars.append(new_character)



#==============apply the damage calculation to each character===================#
dataset = []
for x_jeff in chars:
	mul, dmg, x_jeff = x_jeff.get_damage()
	dataset.append({'factor':round(mul,4), 'damage':round(dmg,2), 'obj':x_jeff})

# and sort ascending
dataset.sort(key=lambda x: x['damage'])


#=========================save to file best N entries===========================#
N = 4
with open(file_path, "w") as file:
    file.write(f'Latest update: {datetime.now()}\n')
    
    for d in dataset[-1:-N:-1]:
        output_string = f"{d['damage']:<10}{d['factor']:<10}\n{d['obj']}\n"
        
        file.write(output_string.replace('|0.0','|   '))

In [2]:

d = dataset[-1]
print(f"{d['damage']:<10}{d['factor']:<10}\n{d['obj']}")

d = dataset[0]
print(f"{d['damage']:<10}{d['factor']:<10}\n{d['obj']}")

37732.92  7.861     
[id]      |name                      |set       |phy_atk   |mag_atk   |phy_def   |mag_def   |hp        |dmg_per   |dmg_per_m |maxi      |crit      |crit_dmg  |pola      |specific  |all_s_dmg |boss_dmg  |adapt     |cdr       |dmg_with_hp
base      |class base stats          |          |1202      |1202      |301       |301       |157000    |18        |0         |0         |25        |0         |0         |0         |3         |0         |0         |0         |0         
HH        |unique HH                 |          |0         |0         |0         |0         |0         |-24       |0         |0         |0         |0         |0         |0         |0         |80        |0         |0         |0         
         3|tene top                  |with hp   |1457      |0         |2185      |2036      |82761     |2.0       |0         |0         |0         |6         |0         |10        |0         |0         |2         |0         |2.0       
         4|tene bottom            